In [4]:
# %%
# ==================================================
# Senado Scraper – Versão Notebook (SEM BD)
# ==================================================

import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time
import plotly.express as px
import plotly.io as pio
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Gráficos interativos
pio.renderers.default = "notebook_connected"

HEADERS = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0 Safari/537.36"
}

# Selenium (headless)
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=chrome_options
)

print("Pronto para scraping do Senado!")


Pronto para scraping do Senado!


In [ ]:

def carregar_pagina_selenium(url):
    """Abre página com Selenium e retorna BeautifulSoup."""
    driver.get(url)
    time.sleep(2)
    return BeautifulSoup(driver.page_source, "html.parser")


def extrair_paragrafos(soup):
    """Extrai parágrafos relevantes da notícia."""
    paragrafos = []
    for p in soup.find_all("p"):
        txt = p.get_text(strip=True)
        if txt and (len(txt.split()) >= 12 or txt.endswith(".")):
            paragrafos.append(txt)
    return paragrafos or ["NA"]


In [10]:
def montar_url(pagina):
    if pagina == 1:
        return "https://www12.senado.leg.br/noticias/ultimas"
    return f"https://www12.senado.leg.br/noticias/ultimas/{pagina}"


In [ ]:
import requests
from bs4 import BeautifulSoup
import time

noticias = []

for pagina in range(5, 0, -1):
    url = montar_url(pagina)
    print(f"Coletando página {pagina}: {url}")

    r = requests.get(url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    lista = soup.find("ol", class_="lista-resultados")
    if not lista:
        print("  Nenhuma lista encontrada")
        continue

    itens = lista.find_all("li")
    print(f"  {len(itens)} notícias encontradas")

    for item in itens:
        titulo_tag = item.find("span", class_="eta")
        link_tag = item.find("a")
        data_tag = item.select_one("div.text-muted.normalis")

        if not titulo_tag or not link_tag:
            continue

        noticias.append({
            "titulo": titulo_tag.get_text(strip=True),
            "link": "https://www12.senado.leg.br" + link_tag["href"],
            "data": data_tag.get_text(strip=True) if data_tag else None
        })

    time.sleep(1)

print(f"\nTotal coletado: {len(noticias)} notícias")


Coletando página 5: https://www12.senado.leg.br/noticias/ultimas/5
  20 notícias encontradas
Coletando página 4: https://www12.senado.leg.br/noticias/ultimas/4
  20 notícias encontradas
Coletando página 3: https://www12.senado.leg.br/noticias/ultimas/3
  20 notícias encontradas
Coletando página 2: https://www12.senado.leg.br/noticias/ultimas/2
  20 notícias encontradas
Coletando página 1: https://www12.senado.leg.br/noticias/ultimas
  20 notícias encontradas

Total coletado: 94 notícias


In [ ]:

import pandas as pd

df_senado = pd.DataFrame(noticias)

print(df_senado.shape)
df_senado.head()


(94, 3)


,titulo,link,data
0,Avança aproveitamento hidrelétrico em terra in...,https://www12.senado.leg.br/noticias/materias/...,10/12/2025 16h25
1,CDH aprova projeto que reforça habitação adequ...,https://www12.senado.leg.br/noticias/materias/...,10/12/2025 16h22
2,Paim comemora aprovação do fim da escala 6x1 n...,https://www12.senado.leg.br/noticias/materias/...,10/12/2025 16h17
3,CCT aprova 24 outorgas de emissoras de rádio e TV,https://www12.senado.leg.br/noticias/materias/...,10/12/2025 16h10
4,CCJ aprova uso de créditos tributários para ex...,https://www12.senado.leg.br/noticias/materias/...,10/12/2025 15h41


In [ ]:

import re
import pandas as pd
import plotly.express as px

keywords = ['digital', 'internet', 'ia', 'privacidade', 'governo', 'tecnologia']
pattern = r'|'.join(keywords)

# Garantia de string
df_plot = df_senado.copy()
df_plot['titulo'] = df_plot['titulo'].fillna("").astype(str)

# Filtro somente pelo título
df_filt = df_plot[
    df_plot['titulo'].str.contains(pattern, case=False, regex=True)
].copy()

print(f"\n{len(df_filt)} notícias filtradas (de {len(df_plot)})")



45 notícias filtradas (de 94)


In [ ]:
def plot_charts(df):
    if df.empty:
        print("Sem dados para gráficos.")
        return

    top15 = df.head(15).copy()
    top15['rank'] = range(1, len(top15) + 1)

    fig1 = px.bar(
        top15,
        x='rank',
        y='titulo',
        orientation='h',
        title='Top 15 Notícias – Senado'
    )

    fig1.update_layout(
        height=600,
        yaxis={'categoryorder': 'total ascending'}
    )

    fig1.show()

    if 'data' in df.columns:
        df_time = df.copy()
        df_time['data_limpa'] = pd.to_datetime(
            df_time['data'].str.extract(r'(\d{2}/\d{2}/\d{4})')[0],
            dayfirst=True,
            errors='coerce'
        )

        serie = df_time['data_limpa'].value_counts().sort_index().reset_index()
        serie.columns = ['data', 'quantidade']

        fig2 = px.line(
            serie,
            x='data',
            y='quantidade',
            title='Notícias ao longo do tempo'
        )
        fig2.show()

    text = ' '.join(df['titulo']).lower()
    words = re.findall(r'\b[a-záéíóúâêôãõç]{4,}\b', text)

    if words:
        wc = pd.Series(words).value_counts().head(20).reset_index()
        wc.columns = ['palavra', 'freq']

        fig3 = px.treemap(
            wc,
            path=['palavra'],
            values='freq',
            title='Nuvem de Palavras – Títulos'
        )
        fig3.show()


In [22]:
plot_charts(df_filt if not df_filt.empty else df_plot)
